In [2]:
!pip install word2number -q


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pickle
import sqlite3
import pandas as pd

from word2number import w2n



In [3]:

class DataExtractor:
    def __init__(self, invoice_file, expired_invoices_file):
        self.invoice_file = invoice_file
        self.expired_invoices_file = expired_invoices_file
        self.__invoices = None
        self.__expired_invoices = None

    def load_data(self):
        with open(self.invoice_file, 'rb') as file:
            self.__invoices = pickle.load(file)

        with open(self.expired_invoices_file, 'r') as file:
            self.__expired_invoices = file.readlines()

    def transform_data(self):
        
        filtered_data = [dct for dct in self.__invoices if 'items' in dct]
        ids = []
        created_on = []
        invoiceitem_ids = []
        invoiceitem_name = []
        unite_price = []
        types = []
        quantity = []
        
        for inv in filtered_data:
            ids.append(inv['id'])
            created_on.append(inv['created_on'])
            for item in inv['items']:
                invoiceitem_ids.append(item['item']['id'])
                invoiceitem_name.append(item['item']['name'])
                unite_price.append(item['item']['unit_price'])
                types.append(item['item']['type'])
                quantity.append(item['quantity'])
                ids.append(inv['id'])
                created_on.append(inv['created_on'])
                
            ids.pop()
            created_on.pop()
            
        columns = {
            'invoice_id': pd.Series(ids, dtype='object'),
            'created_on': pd.Series(created_on, dtype='object'),
            'invoiceitem_id': pd.Series(invoiceitem_ids, dtype='object'),
            'invoiceitem_name': pd.Series(invoiceitem_name, dtype='str'),
            'type': pd.Series(types, dtype='str'),
            'unit_price': pd.Series(unite_price, dtype='object'),
            'quantity': pd.Series(quantity, dtype='object'),
        }
        
        df = pd.DataFrame(columns)
        df['created_on'] = pd.to_datetime(df['created_on'], errors='coerce')
        
        def conv_to_int(value):
            try:
                value = str(value).replace('O', '0')
                return int(value)
            except ValueError:
                print(f"Error: Cannot convert id of invoice'{value}' to int")
                return value
        
        df['invoice_id'] = df['invoice_id'].apply(conv_to_int)
        
        def conv_txt_num(text):
            try:
                return w2n.word_to_num(text)
            except ValueError:
                return text

        df['quantity'] = df['quantity'].apply(conv_txt_num)
        
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['type'] = df['type'].map({'0': 'Material', '1': 'Equipment', '2': 'Service', '3': 'Other'})
        df['type'] = df['type'].astype(str)
        df['unit_price'] = df['unit_price'].astype(int)
        df['total_price'] = df['unit_price'] * df['quantity']
        sum_total_price = df.groupby('invoice_id')['total_price'].transform('sum')
        df['percentage_in_invoice'] = df['total_price'] / sum_total_price
        df['expired'] = df['invoice_id'].isin(self.__expired_invoices)
      
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id'], ascending=[True, True])

        return df
     
    def invoices(self):
         if self.__invoices is None:
             return self.load_data()
         
         return self.__invoices
    
    def expired_invoices(self):
        return self.expired_invoices()
    
data_extractor = DataExtractor('invoices_new.pkl', 'expired_invoices.txt')
data_extractor.load_data()
data = data_extractor.transform_data()



In [7]:
data.to_csv('invoices_new_dataframe.csv', index=False)